利用 urllib 的 robotparser 模块，我们可以实现网站 Robots 协议的分析。
## 1. Robots 协议
Robots 协议也称作爬虫协议、机器人协议，它的全名叫作网络爬虫排除标准（Robots Exclusion Protocol），用来告诉爬虫和搜索引擎哪些页面可以抓取，哪些不可以抓取。它通常是一个叫作 robots.txt 的文本文件，一般放在网站的根目录下。

当搜索爬虫访问一个站点时，它首先会检查这个站点根目录下是否存在 robots.txt 文件，如果存在，搜索爬虫会根据其中定义的爬取范围来爬取。如果没有找到这个文件，搜索爬虫便会访问所有可直接访问的页面。  
robots.txt  
User-agent: *  
Disallow: /  
Allow: /public/  

这实现了对所有搜索爬虫只允许爬取 public 目录的功能，将上述内容保存成 robots.txt 文件，放在网站的根目录下，和网站的入口文件（比如 index.php、index.html 和 index.jsp 等）放在一起。  

- User-agent: 描述了搜索爬虫的名称，这里将其设置为 * 则代表该协议对任何爬取爬虫有效。比如，我们可以设置：User-agent: Baiduspider  这就代表我们设置的规则对百度爬虫是有效的。如果有多条 User-agent 记录，则就会有多个爬虫会受到爬取限制，但至少需要指定一条。  
    - User-agent: WebCrawler  
    Disallow:  
    User-agent: *  
    Disallow: /  

- Disallow: 指定了不允许抓取的目录，比如上例子中设置为 / 则代表不允许抓取所有页面。
    - User-agent: *  
    Disallow: /private/  
    Disallow: /tmp/  


- Allow: 一般和 Disallow 一起使用，一般不会单独使用，用来排除某些限制。现在我们设置为 /public/，则表示所有页面不允许抓取，但可以抓取 public 目录。

## 2. 爬虫名称
|爬虫名称|企业名称|网站|
|----|----|----|
|BaiduSpider|百度|www.baidu.com|
|Googlebot|谷歌|www.google.com|
|360Spider|360 搜索|www.so.com|
|YodaoBot|有道|www.youdao.com|
|ia_archiver|Alexa|www.alexa.cn|
|Scooter|altavista|www.altavista.com|




## 3. robotparser
了解 Robots 协议之后，我们就可以使用 robotparser 模块来解析 robots.txt 了。该模块提供了一个类 RobotFileParser，它可以根据某网站的 robots.txt 文件来判断一个爬取爬虫是否有权限来爬取这个网页。

该类用起来非常简单，只需要在构造方法里传入 robots.txt 的链接即可。首先看一下它的声明：  
`urllib.robotparser.RobotFileParser(url='')`  
当然，也可以在声明时不传入，默认为空，最后再使用 set_url() 方法设置一下也可。

下面列出了这个类常用的几个方法。
- set_url()：用来设置 robots.txt 文件的链接。如果在创建 RobotFileParser 对象时传入了链接，那么就不需要再使用这个方法设置了。
- read()：读取 robots.txt 文件并进行分析。注意，这个方法执行一个读取和分析操作，如果不调用这个方法，接下来的判断都会为 False，所以一定记得调用这个方法。这个方法不会返回任何内容，但是执行了读取操作。
- parse()：用来解析 robots.txt 文件，传入的参数是 robots.txt 某些行的内容，它会按照 robots.txt 的语法规则来分析这些内容。
- can_fetch()：该方法传入两个参数，第一个是 User-agent，第二个是要抓取的 URL。返回的内容是该搜索引擎是否可以抓取这个 URL，返回结果是 True 或 False。
- mtime()：返回的是上次抓取和分析 robots.txt 的时间，这对于长时间分析和抓取的搜索爬虫是很有必要的，你可能需要定期检查来抓取最新的 robots.txt。
- modified()：它同样对长时间分析和抓取的搜索爬虫很有帮助，将当前时间设置为上次抓取和分析 robots.txt 的时间。

In [1]:
from urllib.robotparser import RobotFileParser

rp = RobotFileParser()
rp.set_url('http://www.jianshu.com/robots.txt')
rp.read()
print(rp.can_fetch('*', 'http://www.jianshu.com/p/b67554025d7d'))
print(rp.can_fetch('*', "http://www.jianshu.com/search?q=python&page=1&type=collections"))

False
False


可以使用 parse() 方法执行读取和分析

In [4]:
from urllib.robotparser import RobotFileParser
from urllib.request import urlopen

rp = RobotFileParser()
rp.parse(urlopen('http://www.jianshu.com/robots.txt').read().decode('utf-8').split('\n'))
print(rp.can_fetch('*', 'http://www.jianshu.com/'))
print(rp.can_fetch('*', "http://www.jianshu.com"))

HTTPError: HTTP Error 403: Forbidden